In [1]:
install.packages("properties")
library(properties)

envProps <- read.properties("../env.properties")

Sys.setenv(
    "AWS_ACCESS_KEY_ID" = envProps$access_key, 
    "AWS_SECRET_ACCESS_KEY" = envProps$secret_key,
    "AWS_DEFAULT_REGION" = 'us-east-1')


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in readLines(file):
“incomplete final line found on '../env.properties'”


Clear the workspace

In [2]:
rm(list = ls())

## SageMaker Processing

## Reinstall numpy to workaround conflict between reticulate and RStudio v4.x due to BLAS library dependency
## The reinstall process takes 5 minutes.
## https://github.com/rstudio/reticulate/issues/1257

In [ ]:
# %%bash
# python_path=$(which python)
# echo $python_path
# sudo --set-home $python_path -m pip install --no-user --force-reinstall --no-binary numpy numpy

In [10]:
library(tidyverse)

data_file <- 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
abalone <- read_csv(file = data_file, col_names = FALSE)
names(abalone) <- c('sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings')
head(abalone)

Warning message:
“‘timedatectl’ indicates the non-existent timezone name ‘n/a’”
Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“‘/etc/localtime’ is not identical to any known timezone file”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Rows: 4177 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): X1
dbl (8): X2, X3, X4, X5, X6, X7, X8, X9

ℹ Use `spec()` to retrieve the full column specification for 

sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


## Imports

In [3]:
suppressWarnings(library(reticulate))
path_to_python <- system("which python", intern = TRUE)
use_python(path_to_python)
sagemaker <- import('sagemaker')

In [5]:
role = sagemaker$get_execution_role()
session = sagemaker$Session()
s3_bucket_name = "my_bucket"
s3_prefix = "use-case/data"

account_id <- session$account_id()
region <- session$boto_region_name

algo_ecr_name_tag = "sagemaker-prebuilt-image-xyz:latest"

In [6]:
container_uri <- paste(account_id, "dkr.ecr", region, "amazonaws.com/", algo_ecr_name_tag, sep=".")
print(container_uri)

[1] "482851446821.dkr.ecr.us-east-1.amazonaws.com/.sagemaker-prebuilt-image-xyz:latest"


In [9]:
row_s3_uri = "<TO FILL IN>"
output_uri <- paste("s3://", s3_bucket_name,"/", s3_prefix, "/output",  sep="")
print(output_uri)

[1] "s3://my_bucket/use-case/data/output"


In [ ]:
processor <- sagemaker$processing$ScriptProcessor(image_uri = container_uri,
                                                   command=list("Rscript"),
                                                   role = role,
                                                   instance_count=1L,
                                                   instance_type="ml.c5.xlarge")
processor$run(
    code="./preprocessing.R",
    job_name=paste("r-processing", as.integer(as.numeric(Sys.time())), sep="-"),
    inputs=[ProcessingInput(source=raw_s3, destination="/opt/ml/processing/input")],
        outputs=[
            ProcessingOutput(
                output_name="csv",
                destination="{}/csv".format(output_uri),
                source="/opt/ml/processing/output"
        ],
)